# Introducción a la librería spaCy
En este *notebook* vamos a describir el uso de la librería `spaCy` para el Procesado de Lenguaje Natural.

In [1]:
import spacy
import pandas as pd
import es_core_news_sm
import en_core_web_md
import re

Cargamos la librería y el modelo de lenguaje para el español. Vemos las principales características de la librería y del modelo.

In [2]:
spacy.info()

{'spacy_version': '3.7.5',
 'location': '/home/vic_263/anaconda3/envs/ia/lib/python3.10/site-packages/spacy',
 'platform': 'Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.31',
 'python_version': '3.10.13',
 'pipelines': {'es_core_news_sm': '3.7.0', 'en_core_web_md': '3.7.1'}}

Comprobamos los modelos de lenguaje instalados (compatible con todas las versiones de `spaCy`)

In [3]:
!python -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.7.5) =================
ℹ spaCy installation:
/home/vic_263/anaconda3/envs/ia/lib/python3.10/site-packages/spacy

NAME              SPACY            VERSION                            
es_core_news_sm   >=3.7.0,<3.8.0   3.7.0   ✔
en_core_web_md    >=3.7.2,<3.8.0   3.7.1   ✔



In [4]:
# nlp = spacy.load("es_core_news_md")
nlp = es_core_news_sm.load()

In [5]:
# spacy.info('es_core_news_md')

### Información del modelo
Podemos ver todas las características del modelo cargado con su atributo `meta` (diccionario)

In [6]:
nlp.meta.keys()

dict_keys(['lang', 'name', 'version', 'description', 'author', 'email', 'url', 'license', 'spacy_version', 'spacy_git_version', 'vectors', 'labels', 'pipeline', 'components', 'disabled', 'performance', 'sources', 'requirements'])

In [7]:
nlp.meta['pipeline']

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

## Arquitectura del modelo
Para cada modelo, `spaCy` tiene un vocabulario de palabras conocidas (*lexemes*) que almacena en un `stringStore` global

Cada modelo tiene un conjunto de lexemas del idioma definidos en su Vocabulario

In [8]:
nlp.vocab

In [9]:
print("\nTamaño vocabulario: ", len(nlp.vocab)) #esta longitud es falsa


Tamaño vocabulario:  420


In [10]:
len(nlp.vocab.strings)

182191

`spaCy` no precargar en memoria todo el vocabulario, sino que carga los lexemas conforme los va necesitando:  
> *To reduce the initial loading time, the lexemes in `nlp.vocab` are no longer loaded on initialization for models with vectors. As you process texts, the lexemes will be added to the vocab automatically, just as in small models without vectors.*

In [11]:
for w in nlp.vocab.vectors:
    _ = nlp.vocab[w]

print("\nTamaño vocabulario: ", len(nlp.vocab))


Tamaño vocabulario:  420


In [12]:
nlp.vocab["ciudad"]

In [13]:
nlp.vocab["ciudad"].text

'ciudad'

In [14]:
[prop for prop in dir(spacy.lexeme.Lexeme) if not prop.startswith('_')]

['check_flag',
 'cluster',
 'flags',
 'has_vector',
 'is_alpha',
 'is_ascii',
 'is_bracket',
 'is_currency',
 'is_digit',
 'is_left_punct',
 'is_lower',
 'is_oov',
 'is_punct',
 'is_quote',
 'is_right_punct',
 'is_space',
 'is_stop',
 'is_title',
 'is_upper',
 'lang',
 'lang_',
 'like_email',
 'like_num',
 'like_url',
 'lower',
 'lower_',
 'norm',
 'norm_',
 'orth',
 'orth_',
 'prefix',
 'prefix_',
 'prob',
 'rank',
 'sentiment',
 'set_attrs',
 'set_flag',
 'shape',
 'shape_',
 'similarity',
 'suffix',
 'suffix_',
 'text',
 'vector',
 'vector_norm',
 'vocab']

In [15]:
nlp.vocab["adiós"].has_vector

False

In [16]:
"jhgsfhgjsf" in nlp.vocab

False

In [17]:
nlp.vocab["jhgsfhgjsf"]

In [18]:
"jhgsfhgjsf" in nlp.vocab

True

In [19]:
nlp.vocab["adiós"].orth

15970597814306712899

In [20]:
nlp.vocab.strings[15970597814306712899]

'adiós'

## Procesado de texto
spaCy ejecuta todos los análisis del texto con una sola instrucción. Esta instrucción ejecuta un *pipeline* (procesado secuencial) que implementa:  

- División en tokens  
- Lematizado
- Análisis gramatical
- Análisis de dependencias
- *Name Entity Recognition* (NER)

In [21]:
# Ejemplo de texto
texto = "La casa de Juan es muy bonita."

Lo primero que hacemos es procesar el texto y generar un objeto de tipo `Doc`

In [22]:
doc = nlp(texto)
type(doc)

spacy.tokens.doc.Doc

In [23]:
[prop for prop in dir(spacy.tokens.doc.Doc) if not prop.startswith('_')]

['cats',
 'char_span',
 'copy',
 'count_by',
 'doc',
 'ents',
 'extend_tensor',
 'from_array',
 'from_bytes',
 'from_dict',
 'from_disk',
 'from_docs',
 'from_json',
 'get_extension',
 'get_lca_matrix',
 'has_annotation',
 'has_extension',
 'has_unknown_spaces',
 'has_vector',
 'is_nered',
 'is_parsed',
 'is_sentenced',
 'is_tagged',
 'lang',
 'lang_',
 'mem',
 'noun_chunks',
 'noun_chunks_iterator',
 'remove_extension',
 'retokenize',
 'sentiment',
 'sents',
 'set_ents',
 'set_extension',
 'similarity',
 'spans',
 'tensor',
 'text',
 'text_with_ws',
 'to_array',
 'to_bytes',
 'to_dict',
 'to_disk',
 'to_json',
 'to_utf8_array',
 'user_data',
 'user_hooks',
 'user_span_hooks',
 'user_token_hooks',
 'vector',
 'vector_norm',
 'vocab']

In [24]:
doc.text

'La casa de Juan es muy bonita.'

In [25]:
type(doc.text)

str

In [26]:
doc

La casa de Juan es muy bonita.

## Exploramos el documento

Al analizar un texto, spaCy lo divide en una lista de `tokens`, que se acceden iterando sobre el objeto `Doc`


In [27]:
[t for t in doc]

[La, casa, de, Juan, es, muy, bonita, .]

In [28]:
len(doc)

8

In [29]:
doc[-1]

.

Esta división es distinta de la simple división por espacios:

In [30]:
texto.split(' ')

['La', 'casa', 'de', 'Juan', 'es', 'muy', 'bonita.']

In [31]:
type(doc[0])

spacy.tokens.token.Token

Cada token tiene una serie de atributos:

In [32]:
print([prop for prop in dir(spacy.tokens.token.Token) if not prop.startswith('_')])

['ancestors', 'check_flag', 'children', 'cluster', 'conjuncts', 'dep', 'dep_', 'doc', 'ent_id', 'ent_id_', 'ent_iob', 'ent_iob_', 'ent_kb_id', 'ent_kb_id_', 'ent_type', 'ent_type_', 'get_extension', 'has_dep', 'has_extension', 'has_head', 'has_morph', 'has_vector', 'head', 'i', 'idx', 'iob_strings', 'is_alpha', 'is_ancestor', 'is_ascii', 'is_bracket', 'is_currency', 'is_digit', 'is_left_punct', 'is_lower', 'is_oov', 'is_punct', 'is_quote', 'is_right_punct', 'is_sent_end', 'is_sent_start', 'is_space', 'is_stop', 'is_title', 'is_upper', 'lang', 'lang_', 'left_edge', 'lefts', 'lemma', 'lemma_', 'lex', 'lex_id', 'like_email', 'like_num', 'like_url', 'lower', 'lower_', 'morph', 'n_lefts', 'n_rights', 'nbor', 'norm', 'norm_', 'orth', 'orth_', 'pos', 'pos_', 'prefix', 'prefix_', 'prob', 'rank', 'remove_extension', 'right_edge', 'rights', 'sent', 'sent_start', 'sentiment', 'set_extension', 'set_morph', 'shape', 'shape_', 'similarity', 'subtree', 'suffix', 'suffix_', 'tag', 'tag_', 'tensor', 't

Cada propiedad tiene dos atributos:  
- `propiedad`: ID único o *hash* que identifica el valor en un diccionario común a todos los Docs (`stringstore`)
- `propiedad_`: valor de la propiedad  

Veamos los atributos de algunos de los tokens:

In [33]:
token=doc[3]
print("TOKEN:", token)
print("original:", token.orth, token.orth_)
print("lowercased:", token.lower, token.lower_)
print("lemma:", token.lemma, token.lemma_)
print("shape:", token.shape, token.shape_)
print("prefix:", token.prefix, token.prefix_)
print("suffix:", token.suffix, token.suffix_)
print("log probability:", token.prob)
print("Brown cluster id:", token.cluster)
print("POS:", token.pos, token.pos_)
print("tag:", token.tag, token.tag_)
print("morphology:", token.morph)
print("Dependency parsing:", token.dep, token.dep_)


TOKEN: Juan
original: 12079884900455715477 Juan
lowercased: 5414139780887730998 juan
lemma: 12079884900455715477 Juan
shape: 10887629174180191697 Xxxx
prefix: 18003918085672799305 J
suffix: 15055317959408037229 uan
log probability: -20.0
Brown cluster id: 0
POS: 96 PROPN
tag: 96 PROPN
morphology: 
Dependency parsing: 426 nmod


Las propiedades más importantes son (https://spacy.io/api/token#attributes):  
* `orth_`: texto del token
* `lemma_`: lema (palabra base)
* `shape_`: forma ortográfica del token
* `pos_`: Part-of-Speech (genérico)
* `tag_`: POS detallado
* `morph`: Análisis morfológico
* `dep_`: Tipo de dependencia del token (análisis de dependencias)

In [34]:
pd.set_option('display.max_colwidth', None)

datos = map(lambda t: {'token': t.orth_,
                       'lema': t.lemma_,
                       'shape': t.shape_,
                       'POS': t.pos_,
                       'Descripción TAG': spacy.explain(t.tag_),
                       'dependencia': t.dep_,
                       'Descripción dep': spacy.explain(t.dep_),
                       'Morfología': t.morph}, doc)

pd.DataFrame(datos)

,token,lema,shape,POS,Descripción TAG,dependencia,Descripción dep,Morfología
0,La,el,Xx,DET,determiner,det,determiner,"(Definite=Def, Gender=Fem, Number=Sing, PronType=Art)"
1,casa,casa,xxxx,NOUN,noun,nsubj,nominal subject,"(Gender=Fem, Number=Sing)"
2,de,de,xx,ADP,adposition,case,case marking,()
3,Juan,Juan,Xxxx,PROPN,proper noun,nmod,modifier of nominal,()
4,es,ser,xx,AUX,auxiliary,cop,copula,"(Mood=Ind, Number=Sing, Person=3, Tense=Pres, VerbForm=Fin)"
5,muy,mucho,xxx,ADV,adverb,advmod,adverbial modifier,()
6,bonita,bonito,xxxx,ADJ,adjective,ROOT,root,"(Gender=Fem, Number=Sing)"
7,.,.,.,PUNCT,punctuation,punct,punctuation,(PunctType=Peri)


### Diferencia entre token y lexema

In [36]:
nlp_en = en_core_web_md.load()
parsedData = nlp_en("I run a long run")
datos = map(lambda t: {'token': t.orth_,
                       'lema': t.lemma_,
                       'shape': t.shape_,
                       'POS': t.pos_,
                       'Morfología': t.morph,
                       'dependencia': t.dep_,
                       'Descripción dep': spacy.explain(t.dep_)},
                        parsedData)

pd.DataFrame(datos)

,token,lema,shape,POS,Morfología,dependencia,Descripción dep
0,I,I,X,PRON,"(Case=Nom, Number=Sing, Person=1, PronType=Prs)",nsubj,nominal subject
1,run,run,xxx,VERB,"(Tense=Pres, VerbForm=Fin)",ROOT,root
2,a,a,x,DET,"(Definite=Ind, PronType=Art)",det,determiner
3,long,long,xxxx,ADJ,(Degree=Pos),amod,adjectival modifier
4,run,run,xxx,NOUN,(Number=Sing),dobj,direct object


In [37]:
parsedData[1]

run

In [38]:
parsedData[4]

run

In [39]:
parsedData[1]==parsedData[4]

False

In [40]:
parsedData[1].orth

12767647472892411841

In [41]:
nlp.vocab.strings[parsedData[1].orth]

'run'

In [42]:
parsedData[1].orth==parsedData[4].orth

True

In [43]:
parsedData[1].pos_

'VERB'

In [44]:
parsedData[4].pos_

'NOUN'

## Análisis gramatical
Los documentos SpaCy también dividen en texto en oraciones (*sentences*) que son objetos del tipo `spacy.tokens.span.Span`. Podemos iterar con el generador `doc.sents` usando `next()`, `list()`, un bucle o con una comprensión de lista.

In [45]:
texto = "Al Sr. Daniel siempre le gustaron las catedrales. La de su ciudad era tan alta, \
que al mirarla desde su pequeña estatura, tenía que torcer el cuello de tal forma que \
le costaba no marearse. Lo que más temía era que sus pies se despegasen de la tierra \
y la Catedral le arrastrara con ella hasta los cielos. Aun así, un espíritu \
aventurero le llevaba cada tarde hasta la Plaza Mayor."
doc = nlp(texto)

In [46]:
doc.text

'Al Sr. Daniel siempre le gustaron las catedrales. La de su ciudad era tan alta, que al mirarla desde su pequeña estatura, tenía que torcer el cuello de tal forma que le costaba no marearse. Lo que más temía era que sus pies se despegasen de la tierra y la Catedral le arrastrara con ella hasta los cielos. Aun así, un espíritu aventurero le llevaba cada tarde hasta la Plaza Mayor.'

In [47]:
frases = doc.sents
frases

In [59]:
next(frases)

La de su ciudad era tan alta, que al mirarla desde su pequeña estatura, tenía que torcer el cuello de tal forma que le costaba no marearse.

In [49]:
type(next(doc.sents))

spacy.tokens.span.Span

In [50]:
for i, sent in enumerate(doc.sents):
    print("Oración {}:\n{}\n".format(i,sent))

Oración 0:
Al Sr. Daniel siempre le gustaron las catedrales.

Oración 1:
La de su ciudad era tan alta, que al mirarla desde su pequeña estatura, tenía que torcer el cuello de tal forma que le costaba no marearse.

Oración 2:
Lo que más temía era que sus pies se despegasen de la tierra y la Catedral le arrastrara con ella hasta los cielos.

Oración 3:
Aun así, un espíritu aventurero le llevaba cada tarde hasta la Plaza Mayor.



Cada sentencia también tiene sus propios atributos, distintos de los de los tokens. Para spaCy las oraciones son objetos de tipo `spacy.tokens.span.Span`

In [51]:
print([prop for prop in dir(spacy.tokens.span.Span) if not prop.startswith('_')])

['as_doc', 'char_span', 'conjuncts', 'doc', 'end', 'end_char', 'ent_id', 'ent_id_', 'ents', 'get_extension', 'get_lca_matrix', 'has_extension', 'has_vector', 'id', 'id_', 'kb_id', 'kb_id_', 'label', 'label_', 'lefts', 'lemma_', 'n_lefts', 'n_rights', 'noun_chunks', 'orth_', 'remove_extension', 'rights', 'root', 'sent', 'sentiment', 'sents', 'set_extension', 'similarity', 'start', 'start_char', 'subtree', 'tensor', 'text', 'text_with_ws', 'to_array', 'vector', 'vector_norm', 'vocab']


Algunas propiedades de `Span` son distintas que las de cada `Token`:

In [52]:
[prop for prop in dir(spacy.tokens.span.Span) if
 not prop.startswith('_') and not prop in dir(spacy.tokens.token.Token)]

['as_doc',
 'char_span',
 'end',
 'end_char',
 'ents',
 'get_lca_matrix',
 'id',
 'id_',
 'kb_id',
 'kb_id_',
 'label',
 'label_',
 'noun_chunks',
 'root',
 'sents',
 'start',
 'start_char',
 'to_array']

In [53]:
frase=next(doc.sents)
frase

Al Sr. Daniel siempre le gustaron las catedrales.

In [54]:
frase.start

0

In [55]:
frase.end

9

In [56]:
[t for t in doc[frase.start:frase.end]]

[Al, Sr., Daniel, siempre, le, gustaron, las, catedrales, .]

In [57]:
frase.root

gustaron

In [58]:
type(frase.root)

spacy.tokens.token.Token

### Ejercicio 1
Obtén la palabra raíz -atributo `root`- de cada oración del texto anterior y muéstrala.  
La respuesta es  
```python  
[gustaron, alta, era, llevaba]
```

In [61]:
frases = doc.sents
[frase.root for frase in frases]

[gustaron, alta, era, llevaba]

### Part of Speech (POS)
La librería `spaCy` determina el tipo gramatical (POS) de cada palabra en nuestro texto. Creamos un diccionario con los distintos POS de nuestro texto de ejemplo, usando el *hash* de cada POS como clave del diccionario:

In [62]:
{w.pos: (w.pos_, spacy.explain(w.pos_)) for w in doc} 

{85: ('ADP', 'adposition'),
 96: ('PROPN', 'proper noun'),
 86: ('ADV', 'adverb'),
 95: ('PRON', 'pronoun'),
 100: ('VERB', 'verb'),
 90: ('DET', 'determiner'),
 92: ('NOUN', 'noun'),
 97: ('PUNCT', 'punctuation'),
 87: ('AUX', 'auxiliary'),
 84: ('ADJ', 'adjective'),
 98: ('SCONJ', 'subordinating conjunction'),
 89: ('CCONJ', 'coordinating conjunction')}

Cada tipo gramatical (POS) se subdivide en distintas etiquetas según su análisis mnorfológico(atributo `morph`).  
Por ejemplo en nuestro texto tenemos los siguientes `tag`:

In [63]:
pd.DataFrame(set([(w.pos_, w.morph) for w in doc]), columns=['POS', 'morph']).sort_values(by='POS')

,POS,morph
36,ADJ,"(Gender=Fem, Number=Sing)"
28,ADJ,"(Gender=Masc, Number=Sing)"
6,ADP,"(Definite=Def, Gender=Masc, Number=Sing, PronType=Art)"
7,ADP,()
27,ADV,()
4,ADV,(Polarity=Neg)
17,ADV,(Degree=Cmp)
33,AUX,"(Mood=Ind, Number=Sing, Person=3, Tense=Imp, VerbForm=Fin)"
16,CCONJ,()
38,DET,"(Definite=Def, Gender=Fem, Number=Sing, PronType=Art)"


### Ejercicio 2
Crea una lista de Python con todas las palabras del texto `doc` que sean del tipo NOMBRE y además sean de género Femenino.  
Ayuda: tendrás que usar una comprensión de lista filtrando mediante una función de búsqueda de texto (o patrón regular) sobre el valor del atributo `morph` 

Podemos extraer el género de todos los sustantivos a partir de su atributo `morph`:

In [64]:
[(w, re.findall('Gender=(\w+)', str(w.morph))) for w in doc if w.pos_=='NOUN']

[(catedrales, ['Fem']),
 (ciudad, ['Fem']),
 (estatura, ['Fem']),
 (cuello, ['Masc']),
 (forma, ['Fem']),
 (temía, ['Fem']),
 (pies, ['Masc']),
 (tierra, ['Fem']),
 (cielos, ['Masc']),
 (Aun, []),
 (espíritu, ['Masc']),
 (tarde, ['Fem'])]

### Análisis de dependencias (dependency parsing)
La librería `spaCy` también analiza las relaciones entre palabras de una frase.

In [65]:
doc = nlp("El perro de Juan se comió mi bocadillo.")
dependencias = [(t.text, t.dep_, spacy.explain(t.dep_)) for t in doc]
pd.DataFrame(list(dependencias), columns=['texto', 'dependencia', 'explicación'])

,texto,dependencia,explicación
0,El,det,determiner
1,perro,nsubj,nominal subject
2,de,case,case marking
3,Juan,nmod,modifier of nominal
4,se,iobj,indirect object
5,comió,ROOT,root
6,mi,det,determiner
7,bocadillo,obj,object
8,.,punct,punctuation


Cada palabra tiene un tipo de dependencia determinado dentro de la frase. 
La lista completa está en https://spacy.io/docs/api/annotation  
La dependencia `root` coincide con el atributo `root` de cada sentencia.

In [66]:
sent=next(doc.sents)
sent.root

comió

Cada raíz tiene una serie de hijos (tokens que dependen gramaticalmente de esa raíz).

In [67]:
list(sent.root.children)

[perro, se, bocadillo, .]

In [68]:
doc[1].head

comió

In [69]:
doc[1].dep_

'nsubj'

Podemos extender el análisis a cada palabra del texto.

In [70]:
for word in sent: 
    print(word, ': ', str(list(word.children)))

El :  []
perro :  [El, Juan]
de :  []
Juan :  [de]
se :  []
comió :  [perro, se, bocadillo, .]
mi :  []
bocadillo :  [mi]
. :  []


Además, cada palabra tiene una palabra de la que depende (`.head`) y unas palabras que dependen de ella (`children`) a izquierda (`.lefts`) y a derecha (`.rights`).

In [71]:
dependencias = map(lambda t: {
    'Palabra': t.orth_,
    'tipo de dependencia': f"{t.dep_} ({spacy.explain(t.dep_)})",
    'HEAD': t.head.orth_},
    doc)

pd.DataFrame(dependencias)

,Palabra,tipo de dependencia,HEAD
0,El,det (determiner),perro
1,perro,nsubj (nominal subject),comió
2,de,case (case marking),Juan
3,Juan,nmod (modifier of nominal),perro
4,se,iobj (indirect object),comió
5,comió,ROOT (root),comió
6,mi,det (determiner),bocadillo
7,bocadillo,obj (object),comió
8,.,punct (punctuation),comió


In [72]:
dependencias = map(lambda token: {
    'dep. izquierdas': [t.orth_ for t in token.lefts],
    'palabra[tipo de dependencia]': f"{token.orth_} [{token.dep_}]",
    'dep.derechas': [t.orth_ for t in token.rights]},
    doc)

pd.DataFrame(dependencias)

,dep. izquierdas,palabra[tipo de dependencia],dep.derechas
0,[],El [det],[]
1,[El],perro [nsubj],[Juan]
2,[],de [case],[]
3,[de],Juan [nmod],[]
4,[],se [iobj],[]
5,"[perro, se]",comió [ROOT],"[bocadillo, .]"
6,[],mi [det],[]
7,[mi],bocadillo [obj],[]
8,[],. [punct],[]


Podemos representar gráficamente las dependencias con el módulo de visualización `displaCy`:

In [73]:
from spacy import displacy

displacy.render(doc, style='dep', jupyter=True, options={'distance':120})

También se pueden obtener los **sintagmas nominales** (*noun phrases*) de la oración. Cada NP tiene un sustantivo como raíz, acompañado ocasionalmente de las palabras que describen el sustantivo.  

In [74]:
chunks = map(lambda chunk: {'NP': chunk.text,
                            'root': chunk.root.text,
                            'Dep.': chunk.root.dep_,
                            'head': chunk.root.head.text},
             doc.noun_chunks)

pd.DataFrame(chunks)

,NP,root,Dep.,head
0,El perro,perro,nsubj,comió
1,Juan,Juan,nmod,perro
2,mi bocadillo,bocadillo,obj,comió


# Búsqueda de patrones de token
Spacy tiene una clase `Matcher` que permite buscar tokens con un patrón definido en los objetos `Doc`.  
Se puede buscar por el texto del token o por los atributos del token.  
Ref: https://spacy.io/usage/rule-based-matching

In [75]:
from spacy.matcher import Matcher

#inicializamos sobre el vocabulario
matcher = Matcher(nlp.vocab)

In [76]:
#definimos un patrón de texto a buscar
patron = [{"TEXT": "iPhone"}, {"TEXT": "X"}] #Patrón: texto 'iPhone' seguido de texto 'X'
matcher.add("iphone_x", [patron])

#procesamos un documento con el patrón
doc = nlp("El iPhone X salió después del iPhone 8, pero nunca sacaron el iPhone 9")

#llamamos al matcher
matches = matcher(doc)

#iteramos sobre los resultados
for match_id, start, end in matches:
    matched_span = doc[start:end]
    print(matched_span.text)

iPhone X


In [77]:
matches #lista de 'matches'

[(1738708750870670527, 1, 3)]

In [78]:
nlp.vocab.strings[1738708750870670527]

'iphone_x'

In [79]:
doc[1:3]

iPhone X

Podemos buscar por atributos del token:

In [80]:
patron = [{"TEXT": "iPhone"}, {"IS_DIGIT": True}] #Patrón: texto 'iPhone' seguido token con la atributo 'IS_DIGIT' a True
matcher.add("iphone_NN", [patron])
#llamamos al matcher
matches = matcher(doc)

#iteramos sobre los resultados
for match_id, start, end in matches:
    matched_span = doc[start:end] #span del match en el documento
    string_id = nlp.vocab.strings[match_id] #identificador del match
    print(f"{string_id}: {matched_span.text}")

iphone_x: iPhone X
iphone_NN: iPhone 8
iphone_NN: iPhone 9


In [81]:
doc = nlp("A mí me gusta el baile y a Pedro le gustaba tocar la trompeta pero no le gusta María y el gusta el iPhone X")

patron = [{"LEMMA": "gustar"}, {"POS": "DET", "OP": "?"}, {"POS": {"REGEX": "NOUN|PROPN"}}]
matcher.add("gustar_nombre", [patron])
#llamamos al matcher
matches = matcher(doc)

#iteramos sobre el resultado
for match_id, start, end in matches:
    matched_span = doc[start:end]
    string_id = nlp.vocab.strings[match_id]
    print(f"{string_id}: {matched_span.text}")

gustar_nombre: gusta el baile
gustar_nombre: gusta María
gustar_nombre: gusta el iPhone
iphone_x: iPhone X


### Ejercicio 3
Crea un nuevo patrón para el lema "gustar" seguido de un verbo

In [82]:
patron = [{"LEMMA": "gustar"}, {"POS": "VERB"}]
matcher.add("gustar_nombre", [patron])
#llamamos al matcher
matches = matcher(doc)

#iteramos sobre el resultado
for match_id, start, end in matches:
    matched_span = doc[start:end]
    string_id = nlp.vocab.strings[match_id]
    print(f"{string_id}: {matched_span.text}")

gustar_nombre: gusta el baile
gustar_nombre: gustaba tocar
gustar_nombre: gusta María
gustar_nombre: gusta el iPhone
iphone_x: iPhone X


### Ejercicio 4
Busca todas las secuencias de texto formadas por nombre seguido de al menos un adjetivo en el texto siguiente.

In [85]:
texto = "En el agua muerta, de una brillantez de estaño, permanecía inmóvil la barca-correo: \
un gran ataúd cargado de personas y paquetes, con la borda casi a flor de agua. La vela triangular, \
con remiendos obscuros, estaba rematada por un guiñapo incoloro que en otros tiempos había sido una \
bandera española destartalada y delataba el carácter oficial de la vieja embarcación."

matcher = Matcher(nlp.vocab)

patron = [{"POS": {"REGEX": "NOUN|PROPN"}}, {"POS": "ADJ", "OP": "+"}]
matcher.add("noun_adjs", [patron])
#llamamos al matcher
matches = matcher(doc)

#iteramos sobre el resultado
for match_id, start, end in matches:
    matched_span = doc[start:end]
    string_id = nlp.vocab.strings[match_id]
    print(f"{string_id}: {matched_span.text}")

# búsqueda de patrones de doc
También existe una clase para buscar secuencias de tokens (objetos tipo `doc`) dentro de un texto procesado. Se usa la clase `PhraseMatcher`, como explica https://spacy.io/usage/rule-based-matching#phrasematcher.  
Es útil para buscar un listado de palabras o frases en un texto.  

In [ ]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)
paises = ["Alemania", "Países Bajos", "Estados Unidos"]
#definimos patrones como objetos 'doc'
patterns = [nlp.make_doc(text) for text in paises] #sólo tokenizamos
matcher.add("Países", patterns)

doc = nlp("El presidente de Estados Unidos, Barack Obama, "
          "visitó al caciller de Alemania en Berlín")
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)


In [ ]:
matcher = PhraseMatcher(nlp.vocab, attr="LEMMA")
colores = ["azul", "amarillo", "rojo", "verde"]
#definimos patrones como objetos 'doc'
patterns = [nlp(text) for text in colores] #hay que ejecutar todo el pipeline
matcher.add("Colores", patterns)

doc = nlp("Me gusta el azul pero los amarillos me parecen todos feos")
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)